# MAS Evaluation Framework Demo

This notebook demonstrates the OTel Capture and Agentic Analytics Converter with a Google ADK Multi-Agent System.

**Architecture:**
- 2 Researchers (parallel) → Writer (sequential) → Critic (refinement loop)

```
┌─────────────────┐   ┌─────────────────┐
│   Researcher 1  │   │   Researcher 2  │
│   (Technical)   │   │   (Business)    │
└────────┬────────┘   └────────┬────────┘
         │                     │
         └──────────┬──────────┘
                    ▼
          ┌─────────────────┐
          │     Writer      │
          └────────┬────────┘
                   │
                   ▼
          ┌─────────────────┐
          │     Critic      │◄──┐
          └────────┬────────┘   │
                   │            │ (refinement loop)
                   └────────────┘
```

## 1. Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install -q google-genai opentelemetry-api opentelemetry-sdk

In [ ]:
# Set your API key
import os
from google.colab import userdata

# Option 1: Colab secrets
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
except:
    pass

# Option 2: Manual input (uncomment if needed)
# os.environ['GOOGLE_API_KEY'] = 'your-api-key-here'

## 2. Import the OTel Capture & Converter Modules

Upload `otel_capture.py` and `agentic_analytics_converter.py` to Colab first.

In [ ]:
# Upload the modules (run this cell and upload the files)
from google.colab import files
print("Upload otel_capture.py and agentic_analytics_converter.py:")
uploaded = files.upload()

In [ ]:
# Import the modules
from otel_capture import OTelCapture, trace_agent
from agentic_analytics_converter import AgenticAnalyticsConverter, convert_otel_to_analytics

print("✅ Modules imported successfully!")

## 3. Initialize OTel Capture (BEFORE Creating Agents)

In [ ]:
# Initialize tracing - MUST be done before any agent calls
capture = OTelCapture(
    output_file="mas_traces.json",
    service_name="mas-research-demo"
)
capture.start()

# Get a tracer for manual spans
tracer = capture.get_tracer("research-mas")

## 4. Define the Google ADK Agents

In [ ]:
from google import genai
from opentelemetry import trace
import concurrent.futures

# Initialize the GenAI client
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))
MODEL = "gemini-2.0-flash-exp"

# Agent definitions with system prompts
AGENTS = {
    "technical_researcher": {
        "role": "Technical Research Specialist",
        "goal": "Find technical details, implementation approaches, and code examples",
        "system_prompt": """You are a Technical Research Specialist. 
Your role is to find technical details, implementation approaches, and code examples.
Be concise but thorough. Focus on technical accuracy."""
    },
    "business_researcher": {
        "role": "Business Research Analyst",
        "goal": "Find market trends, use cases, and business value",
        "system_prompt": """You are a Business Research Analyst.
Your role is to find market trends, use cases, and business value.
Focus on practical applications and real-world impact."""
    },
    "writer": {
        "role": "Technical Writer",
        "goal": "Synthesize research into a coherent, well-structured document",
        "system_prompt": """You are a Technical Writer.
Combine the research from multiple sources into a coherent, well-structured document.
Make it engaging and informative."""
    },
    "critic": {
        "role": "Quality Critic",
        "goal": "Review and suggest improvements to the draft",
        "system_prompt": """You are a Quality Critic.
Review the document and provide specific, actionable feedback.
If the document is good enough, respond with 'APPROVED'.
Otherwise, list the improvements needed."""
    }
}

print("✅ Agent definitions loaded")

In [ ]:
def call_agent(agent_name: str, task: str, context: str = "") -> str:
    """
    Call an agent with OTel tracing.
    """
    agent = AGENTS[agent_name]
    tracer = trace.get_tracer("research-mas")
    
    with tracer.start_as_current_span(
        f"{agent_name}.execute",
        attributes={
            "traceloop.entity.name": agent_name,
            "crewai.agent.role": agent["role"],
            "crewai.agent.goal": agent["goal"],
            "traceloop.entity.input": task[:500],  # Truncate for span
        }
    ) as span:
        try:
            # Build the prompt
            prompt = f"{agent['system_prompt']}\n\nTask: {task}"
            if context:
                prompt += f"\n\nContext:\n{context}"
            
            # Call the Gemini API with nested LLM span
            with tracer.start_as_current_span(
                "llm.generate",
                attributes={
                    "llm.model": MODEL,
                    "llm.request_type": "chat",
                    "gen_ai.request.model": MODEL,
                }
            ) as llm_span:
                response = client.models.generate_content(
                    model=MODEL,
                    contents=prompt
                )
                result = response.text
                
                # Record token usage if available
                if hasattr(response, 'usage_metadata'):
                    llm_span.set_attribute("llm.usage.prompt_tokens", 
                                          getattr(response.usage_metadata, 'prompt_token_count', 0))
                    llm_span.set_attribute("llm.usage.completion_tokens", 
                                          getattr(response.usage_metadata, 'candidates_token_count', 0))
            
            # Record output
            span.set_attribute("traceloop.entity.output", result[:500])
            span.set_attribute("agent.status", "success")
            
            return result
            
        except Exception as e:
            span.set_attribute("agent.status", "error")
            span.set_attribute("agent.error", str(e))
            span.record_exception(e)
            raise

print("✅ Agent call function defined")

## 5. Define the MAS Workflow

In [ ]:
def run_research_mas(topic: str, max_refinements: int = 2) -> dict:
    """
    Run the full MAS workflow:
    1. Two researchers work in parallel
    2. Writer synthesizes results
    3. Critic reviews in a loop until approved
    """
    tracer = trace.get_tracer("research-mas")
    
    with tracer.start_as_current_span(
        "MAS.workflow",
        attributes={
            "traceloop.workflow.name": "research-workflow",
            "workflow.topic": topic,
            "workflow.max_refinements": max_refinements
        }
    ) as workflow_span:
        
        results = {"topic": topic, "iterations": []}
        
        # ========== PHASE 1: Parallel Research ==========
        print("\n📚 Phase 1: Parallel Research")
        with tracer.start_as_current_span(
            "phase.parallel_research",
            attributes={"phase": "research", "parallel": True}
        ):
            research_task = f"Research the topic: {topic}"
            
            # Run researchers in parallel
            with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
                tech_future = executor.submit(
                    call_agent, "technical_researcher", 
                    research_task + " Focus on technical aspects."
                )
                biz_future = executor.submit(
                    call_agent, "business_researcher", 
                    research_task + " Focus on business aspects."
                )
                
                tech_research = tech_future.result()
                biz_research = biz_future.result()
            
            print("  ✅ Technical Researcher: done")
            print("  ✅ Business Researcher: done")
            
            results["technical_research"] = tech_research
            results["business_research"] = biz_research
        
        # ========== PHASE 2: Sequential Writing ==========
        print("\n✍️ Phase 2: Writing")
        with tracer.start_as_current_span(
            "phase.writing",
            attributes={"phase": "writing", "sequential": True}
        ):
            combined_research = f"""Technical Research:
{tech_research}

Business Research:
{biz_research}"""
            
            draft = call_agent(
                "writer",
                f"Write a comprehensive article about: {topic}",
                context=combined_research
            )
            print("  ✅ Writer: draft created")
            results["initial_draft"] = draft
        
        # ========== PHASE 3: Critic Refinement Loop ==========
        print("\n🔄 Phase 3: Critic Refinement Loop")
        with tracer.start_as_current_span(
            "phase.refinement_loop",
            attributes={"phase": "refinement", "loop": True}
        ):
            current_draft = draft
            iteration = 0
            approved = False
            
            while iteration < max_refinements and not approved:
                iteration += 1
                print(f"  🔄 Iteration {iteration}")
                
                with tracer.start_as_current_span(
                    f"refinement.iteration_{iteration}",
                    attributes={"iteration": iteration}
                ):
                    # Critic reviews
                    critique = call_agent(
                        "critic",
                        "Review this document and provide feedback. Say 'APPROVED' if it's good.",
                        context=current_draft
                    )
                    
                    results["iterations"].append({
                        "iteration": iteration,
                        "critique": critique
                    })
                    
                    if "APPROVED" in critique.upper():
                        approved = True
                        print(f"  ✅ Critic: APPROVED")
                    else:
                        print(f"  📝 Critic: Requested changes")
                        # Writer revises based on feedback
                        current_draft = call_agent(
                            "writer",
                            f"Revise the document based on this feedback:\n{critique}",
                            context=current_draft
                        )
                        results["iterations"][-1]["revised_draft"] = current_draft
            
            results["final_draft"] = current_draft
            results["approved"] = approved
            results["total_iterations"] = iteration
        
        workflow_span.set_attribute("workflow.approved", approved)
        workflow_span.set_attribute("workflow.iterations", iteration)
        
        print(f"\n✅ Workflow complete! Approved: {approved}, Iterations: {iteration}")
        return results

print("✅ MAS workflow defined")

## 6. Run the MAS

In [ ]:
# Run the MAS with a sample topic
TOPIC = "The impact of Large Language Models on software development productivity"

print(f"🚀 Starting MAS workflow for topic:\n'{TOPIC}'")
print("="*60)

results = run_research_mas(TOPIC, max_refinements=2)

print("\n" + "="*60)
print("📄 Final Output Preview:")
print("="*60)
print(results["final_draft"][:1000] + "..." if len(results["final_draft"]) > 1000 else results["final_draft"])

## 7. Save OTel Traces to JSON

In [ ]:
# Stop tracing and save to JSON
trace_file = capture.stop_and_save()

print(f"\n📁 Traces saved to: {trace_file}")
print(f"📊 Total spans captured: {len(capture.get_serialized_spans())}")

In [ ]:
# View the raw OTel traces JSON
import json

with open("mas_traces.json", "r") as f:
    raw_traces = json.load(f)

print("=" * 60)
print("📋 RAW OPENTELEMETRY TRACES")
print("=" * 60)
print(f"Metadata: {raw_traces['capture_metadata']}")
print(f"\nFirst 3 spans:")
for span in raw_traces['spans'][:3]:
    print(f"  - {span['name']} ({span['duration_ms']:.0f}ms)")
    print(f"    Attributes: {list(span['attributes'].keys())[:5]}...")

## 8. Convert to Agentic Analytics

In [ ]:
# Convert raw traces to agentic analytics format
analytics = convert_otel_to_analytics("mas_traces.json", "mas_analytics.json")

print("\n" + "=" * 60)
print("📊 AGENTIC ANALYTICS SUMMARY")
print("=" * 60)
print(f"Session ID: {analytics['session_id'][:16]}...")
print(f"Total Duration: {analytics['total_duration_ms']:.0f}ms")
print(f"\nSummary:")
for key, value in analytics['summary'].items():
    print(f"  {key}: {value}")

In [ ]:
# View agents extracted
print("\n" + "=" * 60)
print("🤖 AGENTS DETECTED")
print("=" * 60)
for agent in analytics['agents']:
    print(f"\n  Agent: {agent['name']}")
    if agent.get('role'):
        print(f"    Role: {agent['role']}")
    if agent.get('goal'):
        print(f"    Goal: {agent['goal'][:50]}..." if len(agent.get('goal', '')) > 50 else f"    Goal: {agent.get('goal')}")
    print(f"    LLM Calls: {agent['llm_calls']}")
    print(f"    Token Usage: {agent['token_usage']}")

In [ ]:
# View task flow (hierarchical)
print("\n" + "=" * 60)
print("📋 TASK FLOW (First 10 tasks)")
print("=" * 60)
for task in analytics['task_flow'][:10]:
    indent = "  " if task['parent_id'] else ""
    status_emoji = "✅" if task['status'] == "OK" else "⚠️"
    print(f"{indent}{status_emoji} {task['name']} [{task['span_type']}] - {task['duration_ms']:.0f}ms")
    if task.get('agent'):
        print(f"{indent}   Agent: {task['agent']}")

In [ ]:
# View flow graph structure
print("\n" + "=" * 60)
print("🔗 FLOW GRAPH")
print("=" * 60)
print(f"Nodes: {len(analytics['flow_graph']['nodes'])}")
print(f"Edges: {len(analytics['flow_graph']['edges'])}")

print("\nSample edges:")
for edge in analytics['flow_graph']['edges'][:5]:
    print(f"  {edge['source'][:8]}... --[{edge['edge_type']}]--> {edge['target'][:8]}...")

## 9. Download the JSON Files

In [ ]:
# Download both JSON files
from google.colab import files

print("📥 Downloading JSON files...")
files.download("mas_traces.json")
files.download("mas_analytics.json")
print("✅ Download complete!")

## 10. Full JSON Preview

In [ ]:
# Full analytics JSON
print(json.dumps(analytics, indent=2)[:5000])
print("\n... (truncated for display)")